# Chapter2 Text Mining

## 2.1 웹 크롤링으로 기초 데이터 수집하기

#### 웹 크롤링 라이브러리 사용하기

In [23]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
import requests
from bs4 import BeautifulSoup
import re

#### 페이지의 URL 정보 추출하기

In [25]:
# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://namu.wiki/RecentChanges"

# 사이트의 HTML 구조에 기반하여 크롤링을 수행합니다.
req = requests.get(source_url)
html = req.content
soup = BeautifulSoup(html, 'lxml')
contents_table = soup.find(name="table")
table_body = contents_table.find(name="tbody")
table_rows = table_body.find_all(name="tr")

# a 태그의 href 속성을 리스트로 추출하여 크롤링할 페이지 리스트를 생성합니다.
page_url_base = "https://namu.wiki"
page_urls = []
for index in range(0, len(table_rows)):
    first_td = table_rows[index].find_all('td')[0]
    td_url = first_td.find_all('a')
    if len(td_url) > 0:
        page_url = page_url_base + td_url[0].get('href')
        if 'png' not in page_url:
            page_urls.append(page_url)
        
# 중복 url을 제거합니다.
page_urls = list(set(page_urls))
for page in page_urls[:5]:
    print(page)

AttributeError: 'NoneType' object has no attribute 'find'

#### URL 페이지 정보를 기반으로 크롤링하기

In [26]:
# 최근 변경된 문서 하나를 크롤링합니다.

req = requests.get(page_urls[0])
html = req.content
soup = BeautifulSoup(html, 'lxml')

contents_table = soup.find(name="article")
title = contents_table.find_all('h1')[0]
category= contents_table.find_all('ul')[0]
content_paragraphs = contents_table.find_all(name = "div", attrs = {"class":"wiki-paragraph"})

content_corpus_list = []

for paragraphs in content_paragraphs:
    content_corpus_list.append(paragraphs.text)

content_corpus = "".join(content_corpus_list)

print(title.text)
print("\n")
print(category.text)
print("\n")
print(content_corpus)

AttributeError: 'NoneType' object has no attribute 'find_all'

### 2.1.1 크롤링: 웹 데이터 가져오기

#### BeautifulSoup을 이용해 웹 크롤링하기

In [12]:
contents_table = soup.find(name="article")
contents_table

<article data-v-24bb48eb="" data-v-459c4f00=""><!-- --> <!-- --> <div class="r" data-v-24bb48eb="" data-v-5058474b=""><div class="g" data-v-5058474b=""><!-- --> <!-- --> <a data-v-24bb48eb="" href="/backlink/MEDIABRIDGE%20RED">역링크</a> <a data-v-24bb48eb="" href="/discuss/MEDIABRIDGE%20RED">토론</a> <a data-v-24bb48eb="" href="/edit/MEDIABRIDGE%20RED" rel="nofollow">편집</a> <a data-v-24bb48eb="" href="/history/MEDIABRIDGE%20RED">역사</a> <!-- --> <a data-v-24bb48eb="" href="/acl/MEDIABRIDGE%20RED" rel="nofollow">ACL</a> <!-- --></div></div> <h1 data-v-24bb48eb=""><a data-v-24bb48eb="" href="/w/MEDIABRIDGE%20RED"><!-- -->MEDIABRIDGE RED</a> <!-- --></h1> <!-- --> <!-- --> <!-- --> <!-- --> <div data-v-37c59eae="" data-v-459c4f00=""><p data-v-37c59eae="">
		    해당 문서를 찾을 수 없습니다.
	    </p> <p data-v-37c59eae=""><a data-v-37c59eae="" href="/edit/MEDIABRIDGE%20RED" rel="nofollow">[새 문서 만들기]</a></p> <!-- --></div> <!-- --> <!-- --></article>

In [14]:
category = contents_table.find_all('ul')

In [15]:
# a태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_url_base = "https://namu.wiki"

In [19]:
# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns = ['title', 'category', 'content_text']
df = pd.DataFrame(columns=columns)

# 각 페이지별 '제목', '카테고리', '본문' 정보를 데이터 프레임으로 만듭니다.
for page_url in page_urls:

    # 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
    req = requests.get(page_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    contents_table = soup.find(name="article")
    
    title = contents_table.find_all('h1')[0]
    category = contents_table.find_all('ul')[0]
    content_paragraphs = contents_table.find_all(name="div", attrs={"class":"wiki-paragraph"})
    
    content_corpus_list = []
    
    # 페이지 내 제목 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if title is not None:
        row_title = title.text.replace("\n", " ")
    else:
        row_title = ""
    
    # 페이지 내 본문 정보에서 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if content_paragraphs is not None:
        for paragraphs in content_paragraphs:
            if paragraphs is not None:
                content_corpus_list.append(paragraphs.text.replace("\n", " "))
            else:
                content_corpus_list.append("")
    else:
        content_corpus_list.append("")
        
    # 페이지 내 카테고리정보에서 “분류”라는 단어와 개행 문자를 제거한 뒤 추출합니다. 만약 없는 경우, 빈 문자열로 대체합니다.
    if category is not None:
        row_category = category.text.replace("\n", " ")
    else:
        row_category = ""
    
    # 모든 정보를 하나의 데이터 프레임에 저장합니다.
    row = [row_title, row_category, "".join(content_corpus_list)]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
    
df.head(5)

AttributeError: 'NoneType' object has no attribute 'find_all'